In [1]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 500)
PANDAS_FILE = '../simplicate_cache/hours.pd'
df = pd.read_pickle(PANDAS_FILE)

df = df.query( 'type=="normal"')
df['turnover'] = df.apply(lambda a: (a['hours']+a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)
df['turnover'] = df.apply(lambda a: a['turnover']/2 if a['project_number'] == 'TOR-3' else a['turnover'], axis=1)
df['week'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').isocalendar()[1], axis=1)
df['month'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').month, axis=1)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [2]:
df[-3:]

,employee,organization,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
6348,Stefan Roovers,Sprout Money BV,Slim Beleggen Agile,SM2021,Sprint 98,normal,110.0,Project Management,False,85.0,0.5,2021-03-03,to_forward,0.0,42.5,9,3
6349,Joost Cornelissen,Oberon,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.8,2021-03-03,to_forward,0.0,0.0,9,3
6350,Joost Cornelissen,ThiemeMeulenhoff B.V.,Examenbundel Design Sprint,THIE-20,Design Sprint,normal,125.0,Creative Direction,False,105.0,0.5,2021-03-03,to_forward,0.0,52.5,9,3


## Turnover

In [3]:
def turnover( project, month=None, from_date=None ):
    query = f'project_number=="{project}"'
    if month: 
        query += f' and month=={month}'
    if from_date:
        query += f' and day>="{from_date}"'
    data = df.query(query)
    return data['turnover'].sum()

turnover( 'BAM-1', from_date='2021-02-01')

25821.4

## Invoices

In [4]:
def invoiced(project, month=None ):
    filter = {'project_number':project}
    if month:
        filter['from_date'] = f'2021-0{month}-01'
        filter['until_date'] = f'2021-0{month+1}-01'
    invoices = sim.invoice( filter )
    tot = 0
    for invoice in invoices:
        #print( invoice.get('invoice_number','????'), invoice['total_excluding_vat'], invoice['status'])
        tot += invoice['total_excluding_vat']
    return tot

def last_invoice_date( project ):
    invoices = sim.invoice( {'project_number':project} )
    invoices = sorted( invoices, key=lambda i: i['date'])
    if invoices:
        return datetime.datetime.strptime( invoices[-1]['date'], '%Y-%m-%d').date()
    
#print( invoiced( 'BAM-1'))
print( last_invoice_date( 'SLIM-16' ))

2020-11-30


## All projects, certain month

In [5]:
def active_projects():
    projects = [{'project': project.get('project_number',''),
                 'spent' : project['budget']['hours'].get('value_spent', 0),
                 'invoiced' : project['budget']['total']['value_invoiced']
                }
                for project in sim.project( {'active':True} )]
    return projects
active_projects()[:3]

[{'project': 'IDFA-2', 'spent': 9048.75, 'invoiced': 0},
 {'project': 'ACC-1', 'spent': 98056.25, 'invoiced': 48567.5},
 {'project': 'TOR-3', 'spent': 99141.25, 'invoiced': 1991.87}]

## Onderhanden werk

In [6]:
pd.set_option('display.max_row', 150)

# project['budget']['total']['value_invoiced']
def corrections(project):
    h = df.query( 'project_number=="{project}"' )
    return h['corrections'].sum()
    
def onderhanden():
    return pd.DataFrame( [{'project':project['project'], 
                           'spent':project['spent'],
                           'corr': corrections( project),
                           'inv': project['invoiced'],
                           'OH':project['spent'] + corrections( project) - project['invoiced']} 
                          for project in active_projects()] ).sort_values( by=['OH'])
oh = onderhanden()
oh.drop( oh[(oh.project=='TOR-3')].index, inplace=True)
oh

,project,spent,corr,inv,OH
12,SM2021,"35,190.0",0.0,"44,370.0","-9,180.0"
29,TEX-2,"2,452.5",0.0,"10,047.5","-7,595.0"
42,HAVA-2,"16,173.8",0.0,"21,225.0","-5,051.2"
33,ONC-2,"5,185.0",0.0,"10,200.0","-5,015.0"
23,AME-1,137.5,0.0,"4,457.5","-4,320.0"
24,VOLK-1,"2,920.5",0.0,"7,120.5","-4,200.0"
66,GREE-6,"11,156.2",0.0,"15,000.0","-3,843.8"
15,LEAN-2,0.0,0.0,"3,600.0","-3,600.0"
35,TER-1,495.0,0.0,"3,380.0","-2,885.0"
57,SLIM-9,0.0,0.0,"2,829.2","-2,829.2"


In [7]:
oh['OH'].sum()

201785.28746700002

# Correcties

In [9]:
df[-3:]

,employee,organization,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
6348,Stefan Roovers,Sprout Money BV,Slim Beleggen Agile,SM2021,Sprint 98,normal,110.0,Project Management,False,85.0,0.5,2021-03-03,to_forward,0.0,42.5,9,3
6349,Joost Cornelissen,Oberon,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.8,2021-03-03,to_forward,0.0,0.0,9,3
6350,Joost Cornelissen,ThiemeMeulenhoff B.V.,Examenbundel Design Sprint,THIE-20,Design Sprint,normal,125.0,Creative Direction,False,105.0,0.5,2021-03-03,to_forward,0.0,52.5,9,3


In [11]:
corrections = df.query('corrections < 0')
corrections

,employee,organization,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
89,Gijs Kattenberg,Thales,Thales - SLA,THAL-1,Service Basic,normal,110.0,Service basic laag,True,110.0,6.0,2021-01-04,projectmanager_approved,-2.0,440.0,1,1
122,Paulo Nuno da Cruz Moreno,Avero,BCM Tool - SLA,BCM-1,Service Basic,normal,110.0,Service basic laag,True,110.0,2.8,2021-01-04,projectmanager_approved,-2.0,82.5,1,1
231,Jochem Tijhuis,VVV Texel,Website,TEX-1,Development Sprints 2021,normal,95.0,Back-end Development,True,105.0,7.0,2021-01-05,projectmanager_approved,-6.0,105.0,1,1
278,Fadhlur Zahri,Amsterdam Internet Exchange BV.,AMS-IX SLA,AMS-1,Service Basic,normal,110.0,Service basic laag,True,110.0,1.2,2021-01-05,projectmanager_approved,-1.2,-0.0,1,1
284,Paulo Nuno da Cruz Moreno,Avero,BCM Tool - SLA,BCM-1,Service Basic,normal,110.0,Service basic laag,True,110.0,4.8,2021-01-05,projectmanager_approved,-4.0,82.5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5776,Jeroen Soeteman,VVV Texel,VVV Texel - SLA,TEX-2,Service Plus,normal,90.0,Service plus laag,True,90.0,4.5,2021-02-24,projectmanager_approved,-0.8,337.5,8,2
5779,Fadhlur Zahri,De Volksbank,Eurowijs - SLA,EUR-1,Service (JM),normal,110.0,Service basic laag,True,99.0,1.5,2021-02-24,projectmanager_approved,-1.5,0.0,8,2
5795,Gerben van Dijk,Stichting JobOn,JobOn Strippenkaart,BROE-1,Strippenkaart 4,normal,110.0,Maintenance & Support,False,100.0,0.2,2021-02-24,projectmanager_approved,-0.2,0.0,8,2
5961,Fadhlur Zahri,De Volksbank,Eurowijs - SLA,EUR-1,Service (JM),normal,110.0,Service basic laag,True,99.0,1.2,2021-02-25,projectmanager_approved,-1.2,-0.0,8,2


In [37]:
DATE_FORMAT = '%Y-%m-%d'
lastmonth = (datetime.datetime.today() + datetime.timedelta(days=-30)).strftime(DATE_FORMAT)
a = df.query(f'corrections < 0 and day>="{lastmonth}"').groupby(['organization','project_name']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).sort_values('corrections').query('corrections < -10')
a.reset_index()

,organization,project_name,hours,corrections,turnover
0,VVV Texel,Website,76.0,-76.0,-0.0
1,Oncode,Oncode Community aanpassingen,29.5,-29.5,0.0
2,Oerol,Platform,26.0,-26.0,-0.0
3,BAM,Homestudios Traject 2021,14.5,-11.0,373.5
